In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [4]:
browser.visit(nasa_url)
time.sleep(1)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
try:
    news_title = soup.find('div', class_='content_title').text
except AttributeError as e:
    print(e)

news_title

'InSight Is the Newest Mars Weather Service'

In [7]:
try:
    news_p = soup.find("div", class_="article_teaser_body").text
except AttributeError as e:
    print(e)

news_p

"By collecting data around the clock, NASA's lander will provide unique science about the Martian surface."

In [8]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
time.sleep(1)

In [9]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [10]:
try:
    featured_image = soup.find("a", id="full_image")
    featured_image_url = "https://www.jpl.nasa.gov" + featured_image["data-fancybox-href"]
except AttributeError as e:
    print(e)
    
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17843_ip.jpg'

In [11]:
mars_weather_twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_twitter_url)
time.sleep(1)

In [12]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [13]:
try:
    tweets = soup.select("div.stream-item-header span.FullNameGroup")
    for tweet in tweets:
        if tweet.select("strong.fullname")[0].text == "Mars Weather":
            tweet_content = tweet.parent.parent.parent
            mars_weather = tweet_content.select("div.js-tweet-text-container p.tweet-text")[0].find(text=True)
            mars_weather = mars_weather.splitlines()[0]
            break
except AttributeError as e:
    print(e)
    
mars_weather

'InSight sol 81 (2019-02-17), high -17/2F, low -95/-138F, pressure at 7.23hPa, winds from the WNW at 12 mph gusting to 37.8 mph'

In [14]:
mars_facts_url = "https://space-facts.com/mars/"

In [15]:
mars_facts_df = pd.read_html(mars_facts_url)[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
mars_facts_df.rename(columns={0: "Description", 1: "Value"}, inplace=True)
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
mars_facts_table = mars_facts_df.to_html(index=False)
mars_facts_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [18]:
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemispheres_url)
time.sleep(0.5)

In [19]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [20]:
try:
    img_items = soup.select("div.description")
    hemisphere_image_urls = []

    for item in img_items:
        title = item.find("h3").text

        img_link = "https://astrogeology.usgs.gov" + item.find("a", class_="itemLink product-item")["href"]
        browser.visit(img_link)
        time.sleep(0.5)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        img_url = soup.select_one("div.downloads a")["href"]
        hemisphere_image_urls.append({"title": title, "img_url": img_url})
except AttributeError as e:
    print(e)
        
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]